***Webscraper Hotel Rates Parity - 123 Milhas vs. Decolar***

In [1]:
!pip install selenium
!pip install unidecode
!apt-get update 
!apt install chromium-chromedriver

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graphics-driver

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import *
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from unidecode import unidecode
from time import sleep
import pandas as pd
import os

In [3]:
# Search params

name_destination = 'Florianopolis'
id_destination_123milhas = '1001872'
#id_destination_decolar = ''
checkin = '06-02-2022' # Formato DD-MM-AAAA
checkout = '08-02-2022' # Formato DD-MM-AAAA
pax = 2

In [4]:
# Chromedriver config

chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
chrome_options.add_argument('--lang=pt-BR')
chrome_options.add_argument('--disable-notifications')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [5]:
# Get fee 123 Milhas

def get_fee():
    driver = webdriver.Chrome(options=chrome_options)
    link = f'https://123milhas.com/hoteis?adultos={pax}&bebes=0&checkin={checkin}&checkout={checkout}&criancas=0&id={id_destination_123milhas}&localizacao={name_destination},%20,%20Brasil&quartos={pax}&tipo=City&search_id='
    driver.get(link)
    sleep(3)
       
    price_net_temp = []
    tax_temp = []

    wait = WebDriverWait(driver, 10)

    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'image-slider__main-image')))        
    hotel1_attributes = driver.find_element(By.XPATH, '/html/body/div[1]/div/hotel-card-price-list-and-filter-wrapper/div/div[3]/div[1]/hotel-list-card[1]/div[2]/div[3]/div[4]/theme-button/button').get_attribute('outerHTML')
    attributes_filter1 = ''.join(hotel1_attributes.partition('id')[1:])
    attributes_filter2 = attributes_filter1.find('type')
    attributes_filter3 = attributes_filter1[:attributes_filter2]
    attributes_replacers = attributes_filter3.replace('"','').replace('=','-').replace(' ','').replace('id-','')
    query = "'#" + attributes_replacers + "'"
    
    driver.execute_script(f"document.querySelector({query}).click();")
    new_window = driver.window_handles[1]
    driver.switch_to.window(new_window)
    
    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'rooms-details-and-price__button')))
    driver.execute_script("document.querySelector('#hotel-details-and-info > div > rooms-details-and-price > div > div.rooms-details-and-price__prices-select-holder > theme-button > button').click();")
    
    wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'hotel-checkout-prices-and-tax')))        
    price_net_elem = driver.find_element(By.XPATH, '//*[@id="checkout-form"]/div/div[2]/div[1]/hotel-checkout-prices-and-tax/div/div[2]/span[2]')
    tax_elem = driver.find_element(By.XPATH, '//*[@id="checkout-form"]/div/div[2]/div[1]/hotel-checkout-prices-and-tax/div/div[3]/span[2]')

    price_net_temp.append(price_net_elem.text)
    tax_temp.append(tax_elem.text)
    price_net = rate_fmt(price_net_temp)
    tax = rate_fmt(tax_temp)
    
    global fee
    calc_fee = tax / price_net
    fee = float("%.4f" % calc_fee)
    return fee


def rate_fmt(rate):
    rate = ' '.join(rate)
    rate = rate.strip('R$').lstrip().replace(',','.')
    rate = float(rate)
    return rate



In [6]:
print('Fee:')
get_fee()

Fee:


0.1446

In [7]:
# Scraper Hotel Prices 123 Milhas

list_name_hotels_123milhas = []
list_price_hotels_123milhas = []
list_channel_hotels_123milhas = []

class Scraper123milhas:

    def scraper(self):
        self.data_collect()

    def data_collect(self):
        self.list_names = []
        self.list_prices = []
        self.list_channels = []
        self.driver = webdriver.Chrome(options=chrome_options)
        self.link = f'https://123milhas.com/hoteis?adultos={pax}&bebes=0&checkin={checkin}&checkout={checkout}&criancas=0&id={id_destination_123milhas}&localizacao={name_destination},%20,%20Brasil&quartos={pax}&tipo=City&search_id='
        self.driver.get(self.link)
        sleep(3)
        wait = WebDriverWait(self.driver, 10)

        for p in range(20): # Number of pages to browse (nº > total, run through all)
            item = 1
            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'hotel-list-card__total-price')))
            list_length = self.driver.find_elements(By.CLASS_NAME, 'hotel-list-card__total-price')
            list_max = len(list_length)
            for i in range(list_max):
                c = 1
                while c < list_max:
                    try:
                        names_temp = self.driver.find_elements(By.XPATH,
                            f'//*[@id="hotelListHolder"]/div[3]/div[1]/hotel-list-card[{item}]/div[2]/div[1]/div/h4')
                        self.list_names.append(names_temp[0].text)                       
                        sleep(1)
                        prices_temp = self.driver.find_elements(By.XPATH,
                            f'//*[@id="hotelListHolder"]/div[3]/div[1]/hotel-list-card[{item}]/div[2]/div[2]/div[1]/div/div')
                        self.list_prices.append(prices_temp[0].text)          
                        sleep(1)
                        channels_temp = self.driver.find_element(By.XPATH,
                            f'/html/body/div[1]/div/hotel-card-price-list-and-filter-wrapper/div/div[3]/div[1]/hotel-list-card[{item}]/div[2]/div[1]/image-slider/div/div[1]/img').get_attribute('outerHTML')
                        if len(channels_temp) == 0:
                            channels_temp = self.driver.find_element(By.XPATH, f'/html/body/div[1]/div/hotel-card-price-list-and-filter-wrapper/div/div[3]/div[1]/hotel-list-card[{item}]/div[2]/div[2]/image-slider/div/div[1]/img').get_attribute('outerHTML')                        
                        if 'omnibees' in channels_temp:
                            self.list_channels.append(1)
                        else:
                            self.list_channels.append(0)
                        sleep(1)
                        item += 1
                    except:
                        item += 1
                        c += 1                        
            try:
              wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'pagination-next')))
              self.driver.execute_script('document.querySelectorAll("#hotelListHolder > div.hotel-card-price-list__cards-holder.hotel-card-price-list__obj-align > div.hotel-card-price-list__pagination-container > ul > li.pagination-next > a")[0].click();')
              print(f'\u001b[32m{"Browsing..."}\u001b[0m')
              sleep(3)
            except:
              self.driver.quit()

        print('Raw Data:')
        print(self.list_names)
        print(self.list_prices)
        print(self.list_channels)

        for name in self.list_names:
            list_name_hotels_123milhas.append(name)
        for price in self.list_prices:
            list_price_hotels_123milhas.append(price)  
        for channel in self.list_channels:
            list_channel_hotels_123milhas.append(channel)

start = Scraper123milhas()
start.scraper()


Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Browsing...
Raw Data:
['Hotel Cecomtur Florianópolis - Rede Andrade', 'Hotel Intercity Portofino Florianópolis', 'Hotel Valerim Florianópolis', 'Slaviero Baía Norte Florianópolis', 'Faial Prime Suites', 'Castelmar Hotel', 'Blue Tree Premium Florianópolis', 'Novotel Florianópolis', 'Cris Hotel', 'Village Paraíso', 'Lumar Hotel - Rede Andrade', 'Gentil Hotel', 'Hotel Intercity Florianópolis', 'Pousada Bizkaia', 'Ingleses Praia Hotel', 'Porto da Ilha Hotel', 'Interclass Florianópolis', 'Iate Hotel Centro Florianópolis', 'Rio Branco Apart Hotel', 'Hotel Porto Sol Beach', 'Maria do Mar Hotel', 'Hotel Boutique Quinta das Videiras', 'Oscar Hotel', 'Hotel Palace 1', 'Jurerê Beach Village', 'Apart Hotel Roberto Monteiro', 'Ingleses Park Hotel', 'IL Campanario Villaggio R

In [8]:
# Data cleaning

name_hotels_123m_temp = list(pd.Series(list_name_hotels_123milhas).str.upper())
name_hotels_123m_fmtg = []

for name in name_hotels_123m_temp:
    name_hotels_123m_fmtg.append(unidecode(name))
print(name_hotels_123m_fmtg)

['HOTEL CECOMTUR FLORIANOPOLIS - REDE ANDRADE', 'HOTEL INTERCITY PORTOFINO FLORIANOPOLIS', 'HOTEL VALERIM FLORIANOPOLIS', 'SLAVIERO BAIA NORTE FLORIANOPOLIS', 'FAIAL PRIME SUITES', 'CASTELMAR HOTEL', 'BLUE TREE PREMIUM FLORIANOPOLIS', 'NOVOTEL FLORIANOPOLIS', 'CRIS HOTEL', 'VILLAGE PARAISO', 'LUMAR HOTEL - REDE ANDRADE', 'GENTIL HOTEL', 'HOTEL INTERCITY FLORIANOPOLIS', 'POUSADA BIZKAIA', 'INGLESES PRAIA HOTEL', 'PORTO DA ILHA HOTEL', 'INTERCLASS FLORIANOPOLIS', 'IATE HOTEL CENTRO FLORIANOPOLIS', 'RIO BRANCO APART HOTEL', 'HOTEL PORTO SOL BEACH', 'MARIA DO MAR HOTEL', 'HOTEL BOUTIQUE QUINTA DAS VIDEIRAS', 'OSCAR HOTEL', 'HOTEL PALACE 1', 'JURERE BEACH VILLAGE', 'APART HOTEL ROBERTO MONTEIRO', 'INGLESES PARK HOTEL', 'IL CAMPANARIO VILLAGGIO RESORT', 'WK DESIGN HOTEL FLORIANOPOLIS', 'REDE ANDRADE FLOPH', 'IBIS FLORIANOPOLIS', 'HOTEL CANASVIEIRAS INTERNACIONAL - HCI', 'AL MARE PALACE HOTEL', "QUINTA DA BICA D' AGUA STAYHOTEL.CLUB", 'HOTEL TORRES DA CACHOEIRA', 'CARAVAGGIO PRAIA HOTEL', 'HO

In [9]:
price_hotels_123m_fmtg = []

for price in list_price_hotels_123milhas:
    for dot in ["."]:
        price = price.replace(dot, "")
    price_hotels_123m_fmtg.append(price)
print(price_hotels_123m_fmtg)

['R$ 307', 'R$ 434', 'R$ 349', 'R$ 445', 'R$ 553', 'R$ 297', 'R$ 791', 'R$ 610', 'R$ 535', 'R$ 611', 'R$ 294', 'R$ 554', 'R$ 658', 'R$ 525', 'R$ 1117', 'R$ 605', 'R$ 448', 'R$ 412', 'R$ 691', 'R$ 745', 'R$ 419', 'R$ 1397', 'R$ 373', 'R$ 779', 'R$ 1469', 'R$ 790', 'R$ 747', 'R$ 2400', 'R$ 1025', 'R$ 294', 'R$ 288', 'R$ 330', 'R$ 330', 'R$ 501', 'R$ 1367', 'R$ 310', 'R$ 311', 'R$ 363', 'R$ 379', 'R$ 387', 'R$ 391', 'R$ 399', 'R$ 401', 'R$ 416', 'R$ 420', 'R$ 421', 'R$ 428', 'R$ 483', 'R$ 484', 'R$ 490', 'R$ 494', 'R$ 501', 'R$ 570', 'R$ 571', 'R$ 572', 'R$ 575', 'R$ 581', 'R$ 617', 'R$ 660', 'R$ 677', 'R$ 703', 'R$ 708', 'R$ 880', 'R$ 882', 'R$ 947', 'R$ 1025', 'R$ 1146', 'R$ 325', 'R$ 342', 'R$ 343', 'R$ 440', 'R$ 496', 'R$ 503', 'R$ 563', 'R$ 624', 'R$ 699', 'R$ 770', 'R$ 915', 'R$ 1094', 'R$ 301', 'R$ 342', 'R$ 352', 'R$ 462', 'R$ 624', 'R$ 668', 'R$ 737', 'R$ 789', 'R$ 844', 'R$ 880', 'R$ 987', 'R$ 1205', 'R$ 1253', 'R$ 223', 'R$ 342', 'R$ 363', 'R$ 373', 'R$ 421', 'R$ 445', 'R$ 490'

In [10]:
price_hotels_123m_fmtg2 = []

for price in price_hotels_123m_fmtg:
    for sign in ["R$ "]:
        price = price.replace(sign, "")
    price_hotels_123m_fmtg2.append(price)
print(price_hotels_123m_fmtg2)

['307', '434', '349', '445', '553', '297', '791', '610', '535', '611', '294', '554', '658', '525', '1117', '605', '448', '412', '691', '745', '419', '1397', '373', '779', '1469', '790', '747', '2400', '1025', '294', '288', '330', '330', '501', '1367', '310', '311', '363', '379', '387', '391', '399', '401', '416', '420', '421', '428', '483', '484', '490', '494', '501', '570', '571', '572', '575', '581', '617', '660', '677', '703', '708', '880', '882', '947', '1025', '1146', '325', '342', '343', '440', '496', '503', '563', '624', '699', '770', '915', '1094', '301', '342', '352', '462', '624', '668', '737', '789', '844', '880', '987', '1205', '1253', '223', '342', '363', '373', '421', '445', '490', '504', '540', '542', '574', '651', '654', '689', '809', '833', '490', '504', '540', '542', '574', '651', '654', '689', '809', '833', '490', '504', '540', '542', '574', '651', '654', '689', '809', '833', '490', '504', '540', '542', '574', '651', '654', '689', '809', '833', '490', '504', '540', '

In [11]:
price_hotels_123m_fmtg2 = pd.to_numeric(price_hotels_123m_fmtg2, errors='coerce', downcast='unsigned')

In [12]:
data_hoteis_123m = []

for elems in zip(name_hotels_123m_fmtg, price_hotels_123m_fmtg2):
    data_hoteis_123m.append(elems)
data_hoteis_123m_dict = dict(data_hoteis_123m)
print(data_hoteis_123m_dict)

{'HOTEL CECOMTUR FLORIANOPOLIS - REDE ANDRADE': 307, 'HOTEL INTERCITY PORTOFINO FLORIANOPOLIS': 434, 'HOTEL VALERIM FLORIANOPOLIS': 349, 'SLAVIERO BAIA NORTE FLORIANOPOLIS': 445, 'FAIAL PRIME SUITES': 553, 'CASTELMAR HOTEL': 297, 'BLUE TREE PREMIUM FLORIANOPOLIS': 791, 'NOVOTEL FLORIANOPOLIS': 610, 'CRIS HOTEL': 535, 'VILLAGE PARAISO': 611, 'LUMAR HOTEL - REDE ANDRADE': 294, 'GENTIL HOTEL': 554, 'HOTEL INTERCITY FLORIANOPOLIS': 658, 'POUSADA BIZKAIA': 525, 'INGLESES PRAIA HOTEL': 1117, 'PORTO DA ILHA HOTEL': 605, 'INTERCLASS FLORIANOPOLIS': 448, 'IATE HOTEL CENTRO FLORIANOPOLIS': 412, 'RIO BRANCO APART HOTEL': 691, 'HOTEL PORTO SOL BEACH': 745, 'MARIA DO MAR HOTEL': 419, 'HOTEL BOUTIQUE QUINTA DAS VIDEIRAS': 1397, 'OSCAR HOTEL': 373, 'HOTEL PALACE 1': 779, 'JURERE BEACH VILLAGE': 1469, 'APART HOTEL ROBERTO MONTEIRO': 790, 'INGLESES PARK HOTEL': 747, 'IL CAMPANARIO VILLAGGIO RESORT': 2400, 'WK DESIGN HOTEL FLORIANOPOLIS': 1025, 'REDE ANDRADE FLOPH': 294, 'IBIS FLORIANOPOLIS': 288, 'HOTE

In [13]:
df_data_hoteis_123m = pd.DataFrame(list(data_hoteis_123m_dict.items()),
                   columns=['Hotel Name', 'Net Price 123 Milhas'])
display(df_data_hoteis_123m)

,Hotel Name,Net Price 123 Milhas
0,HOTEL CECOMTUR FLORIANOPOLIS - REDE ANDRADE,307
1,HOTEL INTERCITY PORTOFINO FLORIANOPOLIS,434
2,HOTEL VALERIM FLORIANOPOLIS,349
3,SLAVIERO BAIA NORTE FLORIANOPOLIS,445
4,FAIAL PRIME SUITES,553
...,...,...
103,KASATO MARU APART HOTEL,651
104,POUSADA ESTRELA GUIA,654
105,MARATEA MARE,689
106,MORADA PONTA DAS CANAS,809


In [14]:

df_data_hoteis_123m['Total Price 123 Milhas (w/ fee)'] = (df_data_hoteis_123m['Net Price 123 Milhas'] * fee) + df_data_hoteis_123m['Net Price 123 Milhas'].apply(pd.to_numeric) 

In [15]:
df_data_hoteis_123m['Total Price 123 Milhas (w/ fee)'] = df_data_hoteis_123m['Total Price 123 Milhas (w/ fee)'].astype(int)

In [16]:
df_data_hoteis_123m

,Hotel Name,Net Price 123 Milhas,Total Price 123 Milhas (w/ fee)
0,HOTEL CECOMTUR FLORIANOPOLIS - REDE ANDRADE,307,351
1,HOTEL INTERCITY PORTOFINO FLORIANOPOLIS,434,496
2,HOTEL VALERIM FLORIANOPOLIS,349,399
3,SLAVIERO BAIA NORTE FLORIANOPOLIS,445,509
4,FAIAL PRIME SUITES,553,632
...,...,...,...
103,KASATO MARU APART HOTEL,651,745
104,POUSADA ESTRELA GUIA,654,748
105,MARATEA MARE,689,788
106,MORADA PONTA DAS CANAS,809,925


In [17]:
length_df1 = len(df_data_hoteis_123m)
length_df2 = len(list_channel_hotels_123milhas)
diff_length_dfs = length_df2 - length_df1
list_channel_hotels_123milhas = list_channel_hotels_123milhas[:length_df2 - diff_length_dfs]

In [18]:
df_data_hoteis_123m.insert(loc=1, column='Omnibees Provider 123 Milhas', value=list_channel_hotels_123milhas)

In [19]:
df_data_hoteis_123m

,Hotel Name,Omnibees Provider 123 Milhas,Net Price 123 Milhas,Total Price 123 Milhas (w/ fee)
0,HOTEL CECOMTUR FLORIANOPOLIS - REDE ANDRADE,0,307,351
1,HOTEL INTERCITY PORTOFINO FLORIANOPOLIS,1,434,496
2,HOTEL VALERIM FLORIANOPOLIS,0,349,399
3,SLAVIERO BAIA NORTE FLORIANOPOLIS,1,445,509
4,FAIAL PRIME SUITES,0,553,632
...,...,...,...,...
103,KASATO MARU APART HOTEL,0,651,745
104,POUSADA ESTRELA GUIA,0,654,748
105,MARATEA MARE,0,689,788
106,MORADA PONTA DAS CANAS,0,809,925


In [21]:
copy_df_123m = df_data_hoteis_123m.copy()